In [1]:
from Custom_Tetris_Env.tetris import TetrisEnv
from Custom_Tetris_Env.jp import JoypadSpace
from Custom_Tetris_Env.actions import MOVEMENT
from stable_baselines3 import DQN
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold, CheckpointCallback
from stable_baselines3.common.monitor import Monitor


# Separate evaluation env
eval_env = TetrisEnv(reward_lines=True, reward_score=True, penalize_height=True)
eval_env = JoypadSpace(eval_env, MOVEMENT)
# Stop training when the model reaches the reward threshold
callback_on_best = StopTrainingOnRewardThreshold(reward_threshold=200, verbose=1)
eval_callback_stop = EvalCallback(eval_env, callback_on_new_best=callback_on_best, verbose=1)

eval_callback_save = EvalCallback(eval_env, best_model_save_path="./logs/",
                             log_path="./logs/", eval_freq=500,
                             deterministic=True, render=False)
checkpoint = CheckpointCallback(save_freq=50000, save_path="./logs/",
                                name_prefix="dqn_tetris")




env = TetrisEnv(reward_lines=True, reward_score=True, penalize_height=True)
env = JoypadSpace(env, MOVEMENT)
env = Monitor(env)

model = DQN("MlpPolicy", env, verbose=1, tensorboard_log="./tetris_tensorboard/")

/Users/houkinwan/opt/anaconda3/lib/python3.9/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Using cpu device
Wrapping the env in a DummyVecEnv.


AssertionError: The algorithm only supports (<class 'gymnasium.spaces.discrete.Discrete'>,) as action spaces but Discrete(12) was provided

In [2]:
# Train the agent
model.learn(total_timesteps=int(1e5), callback=checkpoint)

Logging to ./tetris_tensorboard/DQN_8


KeyboardInterrupt: 

In [1]:
from stable_baselines3 import DQN
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from Custom_Tetris_Env.tetris import TetrisEnv
from Custom_Tetris_Env.jp import JoypadSpace
from Custom_Tetris_Env.actions import MOVEMENT

# env = TetrisEnv(reward_lines=True, reward_score=True, penalize_height=True)
# env = JoypadSpace(env, MOVEMENT)

def make_custom_env(env_id, rank, seed=0):
    """
    Utility function for multiprocessed env.
    
    :param env_id: (str) the environment ID
    :param rank: (int) index of the subprocess
    :param seed: (int) the initial seed for RNG
    """
    def _init():
        env = TetrisEnv(reward_lines=True, reward_score=True, penalize_height=True)
        env = JoypadSpace(env, MOVEMENT)
        env.seed(seed + rank)
        return env
    return _init

num_cpu = 4  # Number of processes to use
envs = [make_custom_env('YourEnvID', i) for i in range(num_cpu)]  # Replace 'YourEnvID' as needed
vec_env = SubprocVecEnv(envs)

model = DQN.load("Tetris AI with RL.zip", vec_env, verbose=1, tensorboard_log="./tetris_tensorboard/", train_feq=16, batch_size=64, learning_start=0)

ValueError: Error: the model does not support multiple envs; it requires a single vectorized environment.

In [8]:
done = False
obs = env.reset()


while not done:
    action, _states = model.predict(obs)
    obs, reward, done, info = env.step(action)
    env.render()

In [9]:
env.reset()

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.])